In [1]:
from mr_eval.utils.utils import *
from datasets import load_dataset
from collections import Counter
import random
from tqdm import tqdm

/mnt/petrelfs/songmingyang/anaconda3/envs/smoe/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [9]:
def extract_answer_from_model_response(model_response):
    # 使用正则表达式匹配 \boxed{内容}，允许内容中包含大括号
    stack = []
    start = None
    for i, char in enumerate(model_response):
        if i < 7:
            continue
        if model_response[i-7:i] == '\\boxed{':
            stack.append('{')
            if start is None:
                start = i
        elif char == '{' and stack:
            stack.append('{')
        elif char == '}' and stack:
            stack.pop()
            if not stack:
                return model_response[start:i]
    return None


def get_most_common(lst):
    # 使用Counter统计每个元素出现次数
    count = Counter(lst)
    
    # 获取最大出现次数
    max_count = max(count.values())
    
    # 找出所有出现次数等于max_count的元素
    most_common = [item for item, freq in count.items() if freq == max_count]
    
    # 如果只有一个元素出现次数最多,返回该元素
    # 如果有多个元素出现次数相同,随机返回一个
    return random.choice(most_common)

def compare_fraction_decimal(str1, str2):
    def convert_to_float(s):
        # 处理分数形式 \frac{a}{b}
        if r'\\frac' in s:
            # 提取分子分母
            num = s[s.find('{')+1:s.find('}')]
            denom = s[s.rfind('{')+1:s.rfind('}')]
            return float(num) / float(denom)
        else:
            # 处理普通数字字符串
            return float(s)
    
    try:
        num1 = convert_to_float(str1)
        num2 = convert_to_float(str2)
        # 使用近似相等来处理浮点数精度问题
        return abs(num1 - num2) < 1e-10
    except:
        return False

def get_clean_str(ans):
    if ans is None:
        return None
    ans = ans.replace("//","").replace(" ","").replace("dfrac","frac")
    ans = ans.replace("\\","")
    ans = ans.replace("dfrac","frac")
    ans = ans.replace("\\left","").replace("\\right","")
    ans = ans.replace("\\begin{matrix}","\\begin{pmatrix}").replace("\\end{matrix}","\\end{pmatrix}")
    ans = ans.replace("\\begin{vmatrix}","\\begin{pmatrix}").replace("\\end{vmatrix}","\\end{pmatrix}")
    ans = ans.replace("\\begin{bmatrix}","\\begin{pmatrix}").replace("\\end{bmatrix}","\\end{pmatrix}")
    ans = ans.replace("\\quad","").replace("\\;","").replace("\\,","")
    ans = ans.replace("^{\\prime}","'")
    ans = ans.replace("\\cdot","*")
    ans = ans.replace("\\times","*")
    ans = ans.replace("{\\rm ","").replace("}","")
    ans = ans.replace("\\text{","").replace("}","")
    return ans


def compare_answer(ans1,ans2):
    if ans1 is None or ans2 is None:
        return False
    assert isinstance(ans1,str) and isinstance(ans2,str), f"ans1: {ans1}, ans2: {ans2}"
    
    if ans1 in ans2 or ans2 in ans1:
        return True
    return False

In [1]:
## MATH

dataset_addr = "/mnt/petrelfs/songmingyang/songmingyang/data/reasoning/gsm8k"
dataset_addr = "/mnt/petrelfs/songmingyang/songmingyang/data/reasoning/MATH"
dataset = load_dataset(dataset_addr,"default")


/mnt/petrelfs/songmingyang/anaconda3/envs/smoe/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


{'problem': 'How many vertical asymptotes does the graph of $y=\\frac{2}{x^2+x-6}$ have?',
 'level': 'Level 3',
 'type': 'Algebra',
 'solution': 'The denominator of the rational function factors into $x^2+x-6=(x-2)(x+3)$. Since the numerator is always nonzero, there is a vertical asymptote whenever the denominator is $0$, which occurs for $x = 2$ and $x = -3$.  Therefore, the graph has $\\boxed{2}$ vertical asymptotes.'}

In [32]:
solution = dataset["test"][2]['solution']
a = extract_answer_from_model_response(solution)
a

'\\dfrac{9}{7}'

In [34]:
solution
test = "\\boxed{\\dfrac{9}{7}}"
a = extract_answer_from_model_response(test)
a

'\\dfrac{9}{7}'

In [3]:
inferenced_data = "/mnt/petrelfs/songmingyang/code/reasoning/MR_Hallucination/mr_eval/bon_eval/scripts/logs/inference_res_full/math.jsonl"
inferenced_data = process_jsonl(inferenced_data)

In [9]:
inferenced_data[3]

{'id': 'math_3',
 'question': 'Evaluate $i^5+i^{-25}+i^{45}$.',
 'answer': 'i',
 'responses': ["So I need to evaluate \\( i^5 + i^{-25} + i^{45} \\), where \\( i \\) is the imaginary unit. I know that \\( i \\) is defined as the square root of -1, so \\( i^2 = -1 \\). But dealing with higher powers of \\( i \\) can be tricky, so I need to find a pattern or a way to simplify these exponents.\n\nFirst, I recall that the powers of \\( i \\) cycle every four exponents. Let's list out the first few powers to see the pattern:\n\n- \\( i^1 = i \\)\n\n- \\( i^2 = -1 \\)\n\n- \\( i^3 = i^2 \\times i = -1 \\times i = -i \\)\n\n- \\( i^4 = (i^2)^2 = (-1)^2 = 1 \\)\n\n- \\( i^5 = i^4 \\times i = 1 \\times i = i \\)\n\n- \\( i^6 = i^4 \\times i^2 = 1 \\times (-1) = -1 \\)\n\nAnd so on. So, the pattern repeats every four powers: \\( i, -1, -i, 1 \\).\n\nKnowing this, I can simplify any positive exponent of \\( i \\) by finding the remainder when the exponent is divided by 4. This is because the powe

In [63]:


total = 0
pass_at_n_cnt = 0
maj_cnt = 0
fail_cases = []
for item in tqdm(inferenced_data):
    ans_str = item["ans_str"]
    responses = item["responses"]
    answer = extract_answer_from_model_response(ans_str)
    answer = get_clean_str(answer)
    total += 1
    ## maj
    candidate_ans = []
    
    for response in responses:
        res_ans = extract_answer_from_model_response(response)
        res_ans = get_clean_str(res_ans)
        candidate_ans.append(res_ans)
    maj_ans = get_most_common(candidate_ans)
    if compare_answer(maj_ans,answer):
        maj_cnt += 1
    
    ## pass@n
    succ_flag = False
    for response in responses:
        res_ans = extract_answer_from_model_response(response)
        res_ans = get_clean_str(res_ans)
        if compare_answer(res_ans,answer):
            pass_at_n_cnt += 1
            succ_flag = True
            break
    if not succ_flag:
        fail_cases.append((ans_str,item))

print(f"maj@n: {maj_cnt/total}, pass@n: {pass_at_n_cnt/total}")
        

100%|██████████| 5000/5000 [00:37<00:00, 133.21it/s]

maj@n: 0.6554, pass@n: 0.8036


In [64]:
fail_cases[0]

('A real number $x$ is in the domain of $g$ if and only if $$(x-3)^2 - (x-8)^2 \\ge 0.$$ Expanding this out and simplifying, we get $$10x - 55\\ge 0;$$ the smallest solution is $x=\\frac{55}{10}=\\boxed{\\frac{11}{2}}$.\n\nAlternatively, once we have the quadratic equation $$(x-3)^2 - (x-8)^2 \\ge 0,$$ instead of expanding it out, we can observe that $(x-3)^2$ is the square of the distance from $x$ to $3$ on the number line, while $(x-8)^2$ is the square of the distance from $x$ to $8$. Thus, $(x-3)^2-(x-8)^2\\ge 0$ is true if $x$ is closer to $8$ than to $3$, which is true if and only if $x\\ge \\frac{8+3}{2} = \\boxed{\\frac{11}{2}}$.',
 {'id': 'math_15',
  'question': 'What is the smallest real number $x$ in the domain of the function $$g(x) = \\sqrt{(x-3)^2-(x-8)^2}~?$$',
  'answer': '\\frac{11',
  'responses': ["So I have this problem: I need to find the smallest real number \\( x \\) in the domain of the function \\( g(x) = \\sqrt{(x-3)^2 - (x-8)^2} \\). Okay, let's break this do

In [66]:
a = extract_answer_from_model_response(fail_cases[0][0])
a

'\\frac{11}{2}'

In [73]:
b = extract_answer_from_model_response(fail_cases[0][1]["responses"][7])
b

'5.5'

In [74]:
print(get_clean_str(a))

frac{11{2


In [60]:
print(b)

4,\ 6,\ 14,\ 15


In [ ]:
def compare_fraction_decimal(str1, str2):
    def convert_to_float(s):
        # 处理分数形式 \frac{a}{b}
        if r'\\frac' in s:
            # 提取分子分母
            num = s[s.find('{')+1:s.find('}')]
            denom = s[s.rfind('{')+1:s.rfind('}')]
            return float(num) / float(denom)
        else:
            # 处理普通数字字符串
            return float(s)
    
    try:
        num1 = convert_to_float(str1)
        num2 = convert_to_float(str2)
        # 使用近似相等来处理浮点数精度问题
        return abs(num1 - num2) < 1e-10
    except:
        return False

In [2]:
### mmlu
dataset_addr = "/mnt/petrelfs/songmingyang/songmingyang/data/reasoning/gsm8k"
dataset_addr = "/mnt/petrelfs/songmingyang/songmingyang/data/reasoning/mmlu"
dataset = load_dataset(dataset_addr,"all")

In [4]:
dataset["test"][0]

{'question': 'Find the degree for the given field extension Q(sqrt(2), sqrt(3), sqrt(18)) over Q.',
 'subject': 'abstract_algebra',
 'choices': ['0', '4', '2', '6'],
 'answer': 1}

In [5]:
inferenced_data = "/mnt/petrelfs/songmingyang/code/reasoning/MR_Hallucination/mr_eval/bon_eval/scripts/logs/inference_res_full/mmlu.jsonl"
inferenced_data = process_jsonl(inferenced_data)

In [7]:
inferenced_data[0]

{'id': 'mmlu_0',
 'question': 'Find the degree for the given field extension Q(sqrt(2), sqrt(3), sqrt(18)) over Q.',
 'answer': '4',
 'responses': ["So I have this problem here: find the degree of the field extension Q(sqrt(2), sqrt(3), sqrt(18)) over Q. Alright, I need to find [Q(sqrt(2), sqrt(3), sqrt(18)) : Q]. I'm a bit rusty on field extensions, but let's try to work through this step by step.\n\nFirst, I know that when we have a field extension like Q(sqrt(2)), the degree is 2 because sqrt(2) is a root of the polynomial x^2 - 2, which is irreducible over Q. Similarly, Q(sqrt(3)) has degree 2 over Q since sqrt(3) is a root of x^2 - 3, which is also irreducible over Q.\n\nNow, in this problem, we have three square roots: sqrt(2), sqrt(3), and sqrt(18). But wait a minute, sqrt(18) can be simplified. sqrt(18) = sqrt(9*2) = 3*sqrt(2). So, sqrt(18) is just 3 times sqrt(2). That means Q(sqrt(2), sqrt(3), sqrt(18)) is actually the same as Q(sqrt(2), sqrt(3), 3*sqrt(2)). But 3*sqrt(2) is 

In [10]:
total = 0
pass_at_n_cnt = 0
maj_cnt = 0
fail_cases = []
for item in tqdm(inferenced_data):
    ans_str = item["ans_str"]
    responses = item["responses"]
    # answer = extract_answer_from_model_response(ans_str)
    # answer = get_clean_str(answer)
    answer = item["answer"]
    total += 1
    ## maj
    candidate_ans = []
    
    for response in responses:
        res_ans = extract_answer_from_model_response(response)
        res_ans = get_clean_str(res_ans)
        candidate_ans.append(res_ans)
    maj_ans = get_most_common(candidate_ans)
    if compare_answer(maj_ans,answer):
        maj_cnt += 1
    
    ## pass@n
    succ_flag = False
    for response in responses:
        res_ans = extract_answer_from_model_response(response)
        res_ans = get_clean_str(res_ans)
        if compare_answer(res_ans,answer):
            pass_at_n_cnt += 1
            succ_flag = True
            break
    if not succ_flag:
        fail_cases.append((ans_str,item))

print(f"maj@n: {maj_cnt/total}, pass@n: {pass_at_n_cnt/total}")

  0%|          | 0/14042 [00:00<?, ?it/s]

100%|██████████| 14042/14042 [03:16<00:00, 71.57it/s]

maj@n: 0.08125623130608176, pass@n: 0.12540948582822958


In [13]:
fail_cases[1]

(1,
 {'id': 'mmlu_3',
  'question': 'Statement 1 | A factor group of a non-Abelian group is non-Abelian. Statement 2 | If K is a normal subgroup of H and H is a normal subgroup of G, then K is a normal subgroup of G.',
  'answer': 'False, False',
  'responses': ['So I\'ve got these two statements here, and I need to figure out whether they\'re true or false. Let\'s take them one at a time.\n\n**Statement 1:** "A factor group of a non-Abelian group is non-Abelian."\n\nOkay, first things first, I need to recall what a factor group is. A factor group, or quotient group, is formed by taking a group G and a normal subgroup N of G. The factor group is denoted by G/N, and its elements are the cosets of N in G.\n\nNow, the question is about the Abelian property. A group is Abelian if its operation is commutative, meaning that for any two elements a and b in the group, a * b = b * a.\n\nSo, the statement is saying that if G is non-Abelian, then G/N is also non-Abelian.\n\nHmm, is that always tr

In [ ]:
### olympiad
dataset_addr = "/mnt/petrelfs/songmingyang/songmingyang/data/reasoning/gsm8k"
dataset_addr = "/mnt/petrelfs/songmingyang/songmingyang/data/reasoning/OlympiadBench"
configs = ['OE_MM_maths_en_COMP', 'OE_MM_maths_zh_CEE', 'OE_MM_maths_zh_COMP', 'OE_MM_physics_en_COMP', 'OE_MM_physics_zh_CEE', 'OE_TO_maths_en_COMP', 'OE_TO_maths_zh_CEE', 'OE_TO_maths_zh_COMP', 'OE_TO_physics_en_COMP', 'OE_TO_physics_zh_CEE', 'TP_MM_maths_en_COMP', 'TP_MM_maths_zh_CEE', 'TP_MM_maths_zh_COMP', 'TP_MM_physics_en_COMP', 'TP_TO_maths_en_COMP', 'TP_TO_maths_zh_CEE', 'TP_TO_maths_zh_COMP', 'TP_TO_physics_en_COMP']
target_item = None
target_list = []
for config in configs:
    dataset = load_dataset(dataset_addr,config)
    for item in dataset["train"]:
        if item["modality"] != "Multimodal":
            target_list.append(item)

len(target_list)


3607

In [14]:
inferenced_data = "/mnt/petrelfs/songmingyang/code/reasoning/MR_Hallucination/mr_eval/bon_eval/scripts/logs/inference_res_full/olympiadbench.jsonl"
inferenced_data = process_jsonl(inferenced_data)

In [24]:
inferenced_data[0]


{'id': 'olympiadbench_0',
 'question': 'Xenia and Sergey play the following game. Xenia thinks of a positive integer $N$ not exceeding 5000. Then she fixes 20 distinct positive integers $a_{1}, a_{2}, \\ldots, a_{20}$ such that, for each $k=1,2, \\ldots, 20$, the numbers $N$ and $a_{k}$ are congruent modulo $k$. By a move, Sergey tells Xenia a set $S$ of positive integers not exceeding 20 , and she tells him back the set $\\left\\{a_{k}: k \\in S\\right\\}$ without spelling out which number corresponds to which index. How many moves does Sergey need to determine for sure the number Xenia thought of?',
 'answer': ['2'],
 'responses': ["So I'm trying to figure out this problem about Xenia and Sergey playing a game. Let me try to understand what's going on here.\n\nXenia thinks of a positive integer N, which is somewhere between 1 and 5000. Then she picks 20 distinct positive integers a1, a2, ..., a20, with the property that for each k from 1 to 20, N and a_k are congruent modulo k. That 

In [ ]:
inferenced_data[0]

In [21]:
for item in inferenced_data:
    idx = item["id"]
    origin_idx = idx.split("_")[1].strip()
    origin_idx = int(origin_idx)
    target_item = target_list[origin_idx]
    assert target_item["question"] == item["question"]
    item["answer"] = target_item["final_answer"]
    item["ans_str"] = target_item["solution"]


In [23]:
write_jsonl(inferenced_data,"/mnt/petrelfs/songmingyang/code/reasoning/MR_Hallucination/mr_eval/bon_eval/scripts/logs/fix_inference_res200/olympiadbench.jsonl")